## Encrypted Reasoning with Responses API

### Environment Setup

In [1]:
# Import required packages
import os
import json
import uuid
from datetime import datetime
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [2]:
# Set up environment variables for Azure OpenAI
AOAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY_REASONING")

### Business Logic Definition

In [3]:
# Define payment processing function with detailed reasoning
def process_credit_card_transaction(card_number, expiry_date, cvv, amount, merchant_id):
    """
    Simulate secure payment gateway that processes credit card transactions.
    
    Returns:
        dict: Transaction result with status, transaction_id, and details
    """
    # Input validation
    if not card_number or len(card_number.replace("-", "").replace(" ", "")) != 16:
        return {
            "status": "DECLINED",
            "reason": "Invalid card number format",
            "transaction_id": None,
            "risk_score": "N/A",
            "risk_factors": []
        }
    
    if not cvv or len(cvv) not in [3, 4]:
        return {
            "status": "DECLINED", 
            "reason": "Invalid CVV",
            "transaction_id": None,
            "risk_score": "N/A",
            "risk_factors": []
        }
    
    # Risk assessment with detailed factors
    risk_factors = []
    risk_points = 0
    
    # Factor 1: Transaction amount
    if amount < 500:
        risk_factors.append("Low amount ($" + str(amount) + ") - minimal risk")
        risk_points += 1
    elif amount < 2000:
        risk_factors.append("Medium amount ($" + str(amount) + ") - moderate risk")
        risk_points += 5
    else:
        risk_factors.append("High amount ($" + str(amount) + ") - elevated risk")
        risk_points += 10
    
    # Factor 2: Card validation
    risk_factors.append("Card number validated via Luhn algorithm - passed")
    risk_factors.append("CVV length validated - passed")
    
    # Factor 3: Merchant reputation (simulated)
    if "mega-store" in merchant_id.lower() or "store" in merchant_id.lower():
        risk_factors.append("Established merchant recognized - low risk")
        risk_points += 1
    else:
        risk_factors.append("Unknown merchant - elevated risk")
        risk_points += 5
    
    # Determine risk score based on points
    if risk_points <= 3:
        risk_score = "low"
    elif risk_points <= 7:
        risk_score = "medium"
    else:
        risk_score = "high"
    
    # Processing transaction
    transaction_id = f"txn_{uuid.uuid4().hex[:12]}"
    
    return {
        "status": "APPROVED",
        "reason": "All validation checks passed",
        "transaction_id": transaction_id,
        "amount": amount,
        "merchant_id": merchant_id,
        "risk_score": risk_score,
        "risk_points": risk_points,
        "risk_factors": risk_factors,
        "processed_at": datetime.now().isoformat(),
        "validation_checks": {
            "card_format": "valid",
            "cvv_format": "valid",
            "expiry_date": "valid"
        }
    }

# Define tool schema
tools = [
    {
        "type": "function",
        "name": "process_credit_card_transaction",
        "description": "Processes a credit card payment securely. Returns detailed risk assessment with specific factors analyzed.",
        "parameters": {
            "type": "object",
            "properties": {
                "card_number": {
                    "type": "string",
                    "description": "The 16-digit credit card number"
                },
                "expiry_date": {
                    "type": "string",
                    "description": "Card expiry date in MM/YY format"
                },
                "cvv": {
                    "type": "string",
                    "description": "The 3 or 4-digit security code"
                },
                "amount": {
                    "type": "number",
                    "description": "The transaction amount to charge"
                },
                "merchant_id": {
                    "type": "string",
                    "description": "The identifier for the merchant"
                }
            },
            "required": ["card_number", "expiry_date", "cvv", "amount", "merchant_id"]
        }
    }
]

print("✓ Payment processing function and tool schema defined")

✓ Payment processing function and tool schema defined


Responses API is set to stateless mode with `store = False` parameter settings.

In [4]:
# Helper function for Responses API calls with encrypted reasoning
def get_model_response(api_client, model_name, current_context, tools_definition, include_reasoning=True):
    """
    Makes a call to the Responses API with optional encrypted reasoning.
    
    Args:
        include_reasoning: If True, includes encrypted reasoning content in response
    """
    params = {
        "model": model_name,
        "input": current_context,
        "tools": tools_definition,
        "reasoning": {"effort": "low"},
        "store": False,
    }
    
    if include_reasoning:
        params["include"] = ["reasoning.encrypted_content"]
    
    return api_client.responses.create(**params)

print("✓ Helper function for Responses API defined")

✓ Helper function for Responses API defined


### Azure OpenAI Client Setup

In [5]:
# Initialise token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

In [6]:
# Initialise Azure OpenAI client
client = AzureOpenAI(  
    azure_endpoint = AOAI_API_BASE,
    azure_ad_token_provider = token_provider,
    api_version = AOAI_API_VERSION,
)

print("✓ Azure OpenAI client initialised")

✓ Azure OpenAI client initialised


### Step 1: Payment Processing with Encrypted Reasoning

In [7]:
# Configure sub-processes for STEP 1
print("=" * 70)
print("STEP 1: Payment Processing with Encrypted Reasoning")
print("=" * 70)

# User request for payment processing
user_request = {
    "role": "user", 
    "content": """Process this payment:
    - Amount: $1,850.00
    - Merchant: 'Electronics-Mega-Store-456'
    - Card: 4111-1111-1111-1111
    - Expiry: 12/27
    - CVV: 123
    
    Assess risk and process if appropriate.
    
    IMPORTANT: Base your analysis ONLY on the data returned by the payment processing function.
    Do not make assumptions about factors not included in the function response."""
}

# First API call with encrypted reasoning
response_1 = get_model_response(
    api_client = client,
    model_name = AOAI_DEPLOYMENT,
    current_context = [user_request],
    tools_definition = tools,
    include_reasoning = True
)

# Extract outputs
reasoning_item = response_1.output[0]
tool_call_item = response_1.output[1]

print(f"\n✓ Encrypted reasoning generated ({len(reasoning_item.encrypted_content)} chars)")
print(f"✓ Tool called: {tool_call_item.name}")

# Execute payment function
args = json.loads(tool_call_item.arguments)
transaction_result = process_credit_card_transaction(
    card_number = args["card_number"],
    expiry_date = args["expiry_date"],
    cvv = args["cvv"],
    amount = args["amount"],
    merchant_id = args["merchant_id"]
)

print(f"✓ Status: {transaction_result['status']}, Risk: {transaction_result['risk_score']}")
print(f"✓ Risk factors analysed: {len(transaction_result['risk_factors'])}")

STEP 1: Payment Processing with Encrypted Reasoning

✓ Encrypted reasoning generated (1124 chars)
✓ Tool called: process_credit_card_transaction
✓ Status: APPROVED, Risk: medium
✓ Risk factors analysed: 4


### Step 2: Reusing Encrypted Reasoning

In [8]:
# Configure sub-processes for STEP 2
print("\n" + "=" * 70)
print("STEP 2: Reusing Encrypted Reasoning")
print("=" * 70)

# Build context following OpenAI pattern
follow_up_context = [
    user_request,
    reasoning_item,
    tool_call_item,
    {
        "type": "function_call_output",
        "call_id": tool_call_item.call_id,
        "output": json.dumps(transaction_result)
    },
    {
        "role": "user",
        "content": """Explain your risk assessment for this transaction.        
        Do not speculate about factors not included in the function response."""
    }
]

# Second API call
response_2 = get_model_response(
    api_client = client,
    model_name = AOAI_DEPLOYMENT,
    current_context = follow_up_context,
    tools_definition = tools,
    include_reasoning = False
)

# Extract explanation
explanation = None
for item in response_2.output:
    if hasattr(item, 'content') and item.content:
        explanation = item.content[0].text
        break

print("\nModel's explanation:")
print(explanation)


STEP 2: Reusing Encrypted Reasoning

Model's explanation:
Here’s how the risk assessment breaks down, based strictly on the data returned by the processing function:

1. Risk Score: “medium”  
   • The function’s built-in scoring placed this transaction at a medium overall risk level.

2. Total Risk Points: 6  
   • The processing logic assigns points for various factors; the sum of those came to 6.

3. Individual Risk Factors:  
   • Medium amount ($1,850.00) – moderate risk (adds points)  
   • Card number validated via Luhn algorithm – passed (no points)  
   • CVV length validated – passed (no points)  
   • Established merchant recognized – low risk (reduces or adds minimal points)

4. Validation Checks (all “valid”):  
   • card_format: valid  
   • cvv_format: valid  
   • expiry_date: valid  

Because the only substantive risk factor was the transaction amount, the system generated a medium risk score (6 points). All other checks—card format, CVV, expiry date, and merchant rec